This notebook aims to produce transport layers that are fit for the model.

Transport modes are among: roads, railways, waterways, maritime, and multimodal. Roads are required. Note that the multimodal is required if there are more than 2 transport modes.

### Input
- For each mode, a `raw_<mode>_edges.geojson` and and a `raw_<mode>_nodes.geoson`, except for multimodal for which there is only a edge layer
- Required attributes :
    - `raw_roads_edges.geojson`: "class" ("primary", "seconday", etc.), "surface" ("paved, "unpaved")
    - `raw_multimodal_edges.geojson`: "multimodes" ("roads-railways", "roads-maritime", etc.)
    - For the countri
- Optional attributes :
    - `raw_<mode>_edges.geojson`: "capacity" (float, max annual tonnage on the edge)
- Other requirements :
    - the location of countries should be placed in the nodes layers
    - concretely, there should be a "name" attribute in `raw_<mode>_nodes.geoson`, with the trigram of the country in the selected node
       
### Output
- For each mode, a `<mode>_edges.geojson` and a `<mode>_nodes.geoson`
- New attributes :
    - `<mode>_edges.geojson` and `<mode>_nodes.geoson`: 'id' (integer)
    - `<mode>_edges.geojson`: 'end1' and 'end2' (integer, id of nodes located at the end of the edges), 'km' (float, length of edge)
    - Note that the end1 and end2 of edges are found with looking at the node of `<mode>_nodes.geoson` which is the closed to the ends of the dege
- Modification on the geometry
    - (optional) `<mode>_nodes.geoson`: if the end1 or end2 nodes are not located at the ends of the edges, they are moved to the ends of the edges
    - (optional) `<mode>_nodes.geoson`: if there are several points very close to each others (distance epsilon), merge them

### Folder path
- input files : in the input_folder, there should be one subfolder per mode, named `<Mode>` with the files
- output : defined by output_folder

### Points to improve
- we could think of not using a `raw_<mode>_nodes.geoson` and generate the nodes directly from the ends of edges
- but we need to say where the countries are

In [7]:
import os
import time
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import shapely.wkt

input_folder = os.path.join('..', '..', '..', 'Cambodia', 'Data', 'Structured')

In [ ]:
projected_crs = 3857

In [2]:
def loadShp(filename):
    gpdf = gpd.read_file(filename)
    gpdf = gpdf[~gpdf['geometry'].isnull()]
    gpdf = gpdf.to_crs(epsg=4326)
    return gpdf

In [3]:
def getEndPointsFromLine(linestring_obj):
    end1Coord = linestring_obj.coords[0]
    end2Coord = linestring_obj.coords[-1]
    return Point(*end1Coord), Point(*end2Coord)

def getIndexClosestPoint(point, df_with_points):
    distList = [point.distance(item) for item in df_with_points['geometry'].tolist()]
    return int(df_with_points.index[distList.index(min(distList))])

def updateLineString(linestring_obj, newEnd1, newEnd2):
    return LineString([newEnd1.coords[0]] + linestring_obj.coords[1:-1]+[newEnd2.coords[0]])

def assignEndpointsAndUpdate(df_links, id_link, df_nodes, update=False):
    p1, p2 = getEndPointsFromLine(df_links.loc[id_link, 'geometry'])
    id_closest_point1 = getIndexClosestPoint(p1, df_nodes)
    id_closest_point2 = getIndexClosestPoint(p2, df_nodes)
    df_links.loc[id_link, 'end1'] = id_closest_point1
    df_links.loc[id_link, 'end2'] = id_closest_point2
    if update:
        df_links.loc[id_link, 'geometry'] = updateLineString(df_links.loc[id_link, 'geometry'], df_nodes.loc[id_closest_point1, 'geometry'], df_nodes.loc[id_closest_point2, 'geometry'])
    return df_links

def assignEndpointsAndUpdateFullDf(df_links, df_nodes, update=False):
    print('Assigning end nodes to linestring')
    if update:
        print('Stag liens to endnodes')
    res = df_links.copy()
    for i in res.index:
        res = assignEndpointsAndUpdate(res, i, df_nodes, update=update)
    res['end1'] = res['end1'].astype(int)
    res['end2'] = res['end2'].astype(int)
    return res

def getAllEndpoints(df_links):
    all_endpoints = [getEndPointsFromLine(item) for item in df_links['geometry']]
    return [item for sublist in all_endpoints for item in sublist]

def mergePoints(df_nodes, df_links, epsilon):
    print("Nb of original nodes:", df_nodes.shape[0])
    
    all_endpoints = getAllEndpoints(df_links)
    print("Nb of endpoints:", len(all_endpoints))
    
    all_points_gpd = gpd.GeoDataFrame({"geometry": df_nodes['geometry'].tolist()+getAllEndpoints(df_links)}, crs={'init': 'epsg:4326'})
    buffered_polygons = gpd.GeoDataFrame({"geometry": all_points_gpd.buffer(distance=epsilon)}, crs={'init': 'epsg:4326'})
    multipolygon = buffered_polygons.unary_union
    centroids_each_polygon = [polygon.centroid for polygon in multipolygon]
    print("Nb of grouped points:", len(centroids_each_polygon))
    
    return gpd.GeoDataFrame({"id":range(len(multipolygon)), "geometry":centroids_each_polygon}, crs={'init': 'epsg:4326'})

def assignEndpointsOneEdge(row, df_nodes):
    p1, p2 = getEndPointsFromLine(row['geometry'])
    id_closest_point1 = getIndexClosestPoint(p1, df_nodes)
    id_closest_point2 = getIndexClosestPoint(p2, df_nodes)
    row['end1'] = id_closest_point1
    row['end2'] = id_closest_point2
    return row

def assignEndpoints(df_links, df_nodes):
    return df_links.apply(lambda row: assignEndpointsOneEdge(row, df_nodes), axis=1)

In [4]:
def loadAndFormatGeojson(transport_mode, nodeedge, subfolder):
    if nodeedge == "nodes":
        nodes = gpd.read_file(os.path.join(subfolder, "raw_"+transport_mode+"_nodes.geojson"))
        if 'index' in nodes.columns:
            nodes = nodes.drop('index', axis=1)
        nodes['id']=range(nodes.shape[0])
        nodes.index = nodes['id']
        nodes.index.name = "index"
        print("There are", nodes.shape[0], "nodes")
        print(nodes.crs)
        return nodes
    
    if nodeedge == "edges":
        edges = gpd.read_file(os.path.join(subfolder, "raw_"+transport_mode+"_edges.geojson"))
        edges = edges[~edges['geometry'].isnull()]
        if 'index' in edges.columns:
            edges = edges.drop('index', axis=1)
        edges['id']=range(edges.shape[0])
        edges['end1'] = None
        edges['end2'] = None
        if "capacity" not in edges.columns:
            edges['capacity'] = None
        edges.index = edges['id']
        edges.index.name = "index"
        print("There are", edges.shape[0], "edges")
        print(edges.crs)
        return edges
    
    
def addKm(edges, crs):
    # Project the layer. Watch out, the CRS should be adapted to the country
    edges['km'] = edges.to_crs({'init': 'epsg:'+str(crs)}).length/1000
    return edges

# Maritime

In [18]:
transport_mode = 'maritime'
version = "current_version"
subfolder = os.path.join(input_folder, transport_mode.capitalize(), version)

nodes = loadAndFormatGeojson(transport_mode, "nodes", subfolder)
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)

edges = addKm(edges, projected_crs)
edges = assignEndpointsAndUpdateFullDf(edges, nodes)

print(nodes.head())
print(edges.head())

nodes.to_file(os.path.join(output_folder, transport_mode+"_nodes.geojson"), driver="GeoJSON")
edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON")

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 22 nodes
{'init': 'epsg:4326'}
There are 19 edges
{'init': 'epsg:4326'}


C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Assigning end nodes to linestring
                                   name special  \
index                                             
0      Sihanoukville international port    port   
1                                  None     ASI   
2                          Cai Mep Port    port   
3                          Laem Chabang    port   
4                                  None     OCE   

                                           geometry  id  
index                                                    
0        POINT (103.516309673649 10.65094281264956)   0  
1          POINT (113.25972836394 19.9766339017673)   1  
2       POINT (107.0258765937671 10.53711717311858)   2  
3       POINT (100.8726788022483 13.07567616414528)   3  
4      POINT (138.5749843338141 -11.10567941915124)   4  
       FID special                                           geometry  id  \
index                                                                       
0      0.0    None  LINESTRING (103.515492084796

C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\geopandas\io\file.py:62: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# Waterways

In [11]:
transport_mode = 'waterways'
version = "current_version"
subfolder = os.path.join(input_folder, transport_mode.capitalize(), version)

nodes = loadAndFormatGeojson(transport_mode, "nodes", subfolder)
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)
edges = addKm(edges, projected_crs)
edges = assignEndpointsAndUpdateFullDf(edges, nodes)

print(nodes.head())
print(edges.head())

nodes.to_file(os.path.join(output_folder, transport_mode+"_nodes.geojson"), driver="GeoJSON")
edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON")

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 20 nodes
{'init': 'epsg:4326'}
There are 19 edges
{'init': 'epsg:4326'}


C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Assigning end nodes to linestring
       id                      name  \
index                                 
0       0          Chong Kneas Port   
1       1      Kampong Chlnang Port   
2       2  Kampong Chlnang Junction   
3       3         Kratie River Port   
4       4           Kratie Junction   

                                          geometry  
index                                               
0      POINT (103.8220192917276 13.26983357773719)  
1      POINT (104.6812144473571 12.26824772755564)  
2        POINT (104.6920055505009 12.271072307162)  
3      POINT (106.0162086075059 12.48460089813051)  
4      POINT (106.0122032737445 12.48335342660554)  
       end1  end2 special  capacity  id  \
index                                     
0         2     0    None  200000.0   0   
1        10     6    None   20000.0   1   
2         6     5    None   20000.0   2   
3         6     4    None   20000.0   3   
4         4     3    None       NaN   4   

                   

C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\geopandas\io\file.py:62: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# Multimodality

In [13]:
transport_mode = 'multimodal'
version = "current_version"
subfolder = os.path.join(input_folder, transport_mode.capitalize(), version)

edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)
edges['km'] = 0.1 #no impact

print(edges.head())

edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON")

There are 13 edges
{'init': 'epsg:4326'}
               multimodes special   capacity  \
index                                          
0          roads-railways    None  1000000.0   
1       railways-maritime           500000.0   
2      roads-maritime-shv          9000000.0   
3         roads-waterways    None   100000.0   
4         roads-waterways    None        NaN   

                                                geometry  id  end1  end2   km  
index                                                                          
0      LINESTRING (104.9165417337968 11.5726478216031...   0  None  None  0.1  
1      LINESTRING (103.5021688860371 10.6428842124675...   1  None  None  0.1  
2      LINESTRING (103.5122193000025 10.6424756999999...   2  None  None  0.1  
3      LINESTRING (104.9209462 11.5830054, 104.921942...   3  None  None  0.1  
4      LINESTRING (105.4653173355818 11.9856826023330...   4  None  None  0.1  


# Railways

In [17]:
transport_mode = 'railways'
version = "current_version"
subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)

nodes = loadAndFormatGeojson(transport_mode, "nodes", subfolder)
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)
edges = addKm(edges, projected_crs)
edges = assignEndpointsAndUpdateFullDf(edges, nodes)

print(nodes.head())
print(edges.head())

nodes.to_file(os.path.join(output_folder, transport_mode+"_nodes.geojson"), driver="GeoJSON")
edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON")

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 10 nodes
{'init': 'epsg:4326'}
There are 10 edges
{'init': 'epsg:4326'}


C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Assigning end nodes to linestring
          name                                     geometry  id
index                                                          
0      station  POINT (104.9165417310334 11.57264782123487)   0
1         None  POINT (102.9310145520781 13.58146756607205)   1
2         None  POINT (104.8218771869167 11.56651961957141)   2
3         None  POINT (104.6969901718489 11.69402940521593)   3
4         None  POINT (102.5439458929634 13.66549038485579)   4
            From_                         To special  capacity  \
index                                                            
0      Phnom Penh              Svay Sisaphon  unused       NaN   
1      Phnom Penh             Preah Sihanouk    used  700000.0   
2           Snoul                Laos Border  unused       NaN   
3       Bat Doeng  Loch Ninh ,Viet Nam Borde  unused       NaN   
4       Bat Doeng  Loch Ninh ,Viet Nam Borde  unused       NaN   

                                                geometr

C:\Users\celian\anaconda3\envs\DisruptSupplyChain\lib\site-packages\geopandas\io\file.py:62: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# Roads

In [14]:
transport_mode = 'roads'
version = "current_version"
subfolder = os.path.join(input_folder, transport_mode.capitalize(), version)

nodes = loadAndFormatGeojson(transport_mode, "nodes", subfolder)
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)
edges = addKm(edges, projected_crs)
edges = assignEndpointsAndUpdateFullDf(edges, nodes)

print(nodes.head())
print(edges.head())

nodes.to_file(os.path.join(output_folder, transport_mode+"_nodes.geojson"), driver="GeoJSON")
edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON")

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 2564 nodes
epsg:4326
There are 3245 edges
epsg:4326


C:\Users\colon\Anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Assigning end nodes to linestring
       id special district  name                    geometry
index                                                       
0       0     LAO     None  None  POINT (102.66566 17.97427)
1       1     THA     None  None  POINT (100.50715 13.75578)
2       2    None     None  None  POINT (106.36375 10.35352)
3       3     VNM     None  None  POINT (106.71016 10.79518)
4       4    None     0706  None  POINT (104.45389 10.42594)
                         name      class  bridge  nb_lanes  surface  id  end1  \
index                                                                           
0      Mai Da Road (St: 2004)  secondary     0.0       NaN  unpaved   0   795   
1      Mai Da Road (St: 2004)  secondary     0.0       NaN  unpaved   1   793   
2      Mai Da Road (St: 2004)  secondary     0.0       NaN  unpaved   2   785   
3      Mai Da Road (St: 2004)  secondary     0.0       NaN  unpaved   3   771   
4                        None   tertiary     0.0     

# Other stuff, may be useful

### Transition from shp to geojson

In [47]:
for edge_node in ["node", "edge"]:    
    transport_mode = 'roads'
    version = "v8"
    subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)

    filename = "raw_"+transport_mode+"_"+edge_node+"s.shp"
    df = gpd.read_file(os.path.join(subfolder, filename))

    version = "current_version"
    subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)
    df.to_file(os.path.join(subfolder, "raw_"+transport_mode+"_"+edge_node+"s.geojson"), driver="GeoJSON")

### Change to CRS 4326

In [76]:
for edge_node in ["node", "edge"]:    
    transport_mode = 'multimodal'
    version = "current_version"
    subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)
    
    filename = "raw_"+transport_mode+"_"+edge_node+"s.shp"
    df = gpd.read_file(os.path.join(subfolder, "raw_"+transport_mode+"_"+edge_node+"s.geojson"))
    
    df = df.to_crs(4326)
    
    df.to_file(os.path.join(subfolder, "raw_"+transport_mode+"_"+edge_node+"s.geojson"), driver="GeoJSON")